In [1]:
import osmnx as ox
import pandana
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline
ox.config(log_console=True)
ox.__version__

'1.1.1'

In [2]:
# Select city (COP, HEL, WAR)
city = 'COP'

if city == 'COP':
    cityname = 'Copenhagen, Denmark'
    minx, miny, maxx, maxy = 12.42000, 55.61000, 12.65000, 55.78000
elif city == 'HEL':
    cityname = 'Helsinki, Finland'
    minx, miny, maxx, maxy = 24.82345, 60.14084, 25.06404, 60.29496
elif city == 'WAR':
    cityname = 'Warsaw, Poland'
    minx, miny, maxx, maxy = 20.79057, 52.09901, 21.31300, 52.38502

In [3]:
# Get graph by geocoding
try:
    graph = ox.graph_from_place(cityname, network_type="walk")

# Get graph based on bbox if geocoding fails (copenhagen has no polygon on nominatim)
except ValueError:
    graph = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type="walk")
    ignore_geocoding = True
    
graph = ox.projection.project_graph(graph, to_crs=3035)

In [ ]:
# Simplify graph to get real intersections only
# (consolidate nodes within 10m from eachother)
graph = ox.simplification.consolidate_intersections(
    # Graph to simplify
    graph,
    # buffer around each node (project the graph beforehand)
    tolerance=10,
    # Get result as graph (False to get nodes only as gdf)
    rebuild_graph=True,
    # Include dead ends
    dead_ends=True,
    # Reconnect the graph
    reconnect_edges=True
)

In [ ]:
# Max time to walk in minutes (no routing to nodes further than this)
walk_time = 15

# Walking speed
walk_speed = 4.5

# Set a uniform walking speed on every edge
for u, v, data in graph.edges(data=True):
    data['speed_kph'] = walk_speed

graph = ox.add_edge_travel_times(graph)

# Extract node/edge GeoDataFrames, retaining only necessary columns (for pandana)
nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]

In [ ]:
# Select pois based on osm tags
tags = {
    'amenity':[
        'cafe',
        'bar',
        'pub',
        'restaurant'
    ],
    'shop':[
        'bakery',
        'convenience',
        'supermarket',
        'mall',
        'department_store',
        'clothes',
        'fashion',
        'shoes'
    ],
    'leisure':[
        'fitness_centre'
    ]
}

# Get amentities from place/bbox
if ignore_geocoding == True:
    amenities = ox.geometries.geometries_from_bbox(
        maxy, miny, maxx, minx,
        tags=tags
    )
else:
    amenities = ox.geometries.geometries_from_place(
        cityname,
        tags=tags
    )
# Project amenities
amenities = amenities.to_crs(epsg=3035)

In [ ]:
# Save as gpkg
#amenities_point = amenities[amenities['geometry'].apply(lambda x: x.type=='Point')]
#amenities_point.to_file('amenities_'+city+'.gpkg', driver='GPKG')

In [ ]:
# Construct the pandana network model
network = pandana.Network(
    node_x=nodes['x'],
    node_y=nodes['y'], 
    edge_from=edges['u'],
    edge_to=edges['v'],
    edge_weights=edges[['travel_time']]
)

# Extract centroids from the amenities' geometries
centroids = amenities.centroid

In [ ]:
# Specify a max travel distance for analysis
# Minutes -> seconds
maxdist = walk_time * 60

# Set the amenities' locations on the network
network.set_pois(
    category='pois',
    maxdist=maxdist,
    maxitems=10,
    x_col=centroids.x, 
    y_col=centroids.y
)

In [ ]:
# calculate travel time to 10 nearest amenities from each node in network
distances = network.nearest_pois(
    distance=maxdist,
    category='pois',
    num_pois=10
)

distances.astype(int).head()

In [ ]:
# plot distance to nearest amenity
fig, ax = ox.plot_graph(
    graph,
    node_size=0,
    edge_linewidth=0.1,
    edge_color='gray',
    show=False,
    close=False
)

sc = ax.scatter(
    x=nodes['x'],
    y=nodes['y'], 
    c=distances[5],
    s=1,
    cmap='viridis_r'
)

ax.set_title(f'Walking time to 5th nearest poi (seconds)')
plt.colorbar(sc, shrink=0.7).outline.set_edgecolor('none')

In [ ]:
# check data
distances.head()

In [ ]:
nodes.head()

In [ ]:
# Get nodes with wgs coords for output
graph_wgs = ox.projection.project_graph(graph, to_crs=4326)
nodes_wgs = ox.graph_to_gdfs(graph_wgs, edges=False)[['x', 'y']]

In [ ]:
# Join travel time info to nodes
walk_access = nodes.join(distances, on='osmid', how='left')
walk_access_wgs = nodes_wgs.join(distances, on='osmid', how='left')

In [ ]:
walk_access_wgs

In [ ]:
# Save as CSV
walk_access_wgs.to_csv('walk_access_'+city+'.csv', encoding='utf-8')